<a href="https://colab.research.google.com/github/HiteshPandharkar/Credit-Card-Fraud-Detection/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import roc_auc_score as ras


import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/fraud csv/creditcard.csv')

dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
data = dataset.copy()

#data.head()

In [ ]:
data.shape

(284807, 31)

In [ ]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=44, criterion='entropy')

In [ ]:
clf.fit(data[data.columns[:-1]], data['Class'])

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=44, verbose=0,
                     warm_start=False)

In [ ]:
model = SelectFromModel(clf, prefit=True)
x = model.transform(data[data.columns[:-1]])

In [ ]:
clf.feature_importances_

array([0.01426102, 0.01553558, 0.01671898, 0.04832222, 0.06290367,
       0.01201394, 0.01081562, 0.02129529, 0.01116674, 0.02657741,
       0.07398904, 0.07935759, 0.10277714, 0.01024879, 0.14729921,
       0.0098386 , 0.06949085, 0.12509097, 0.0332012 , 0.01548061,
       0.01017501, 0.01089009, 0.01059124, 0.00663542, 0.00962035,
       0.0081888 , 0.01112959, 0.00895908, 0.00839863, 0.00902731])

In [ ]:
features = pd.DataFrame(data={'imp': clf.feature_importances_, 'variable': data.columns[:-1]})

features

,imp,variable
0,0.014261,Time
1,0.015536,V1
2,0.016719,V2
3,0.048322,V3
4,0.062904,V4
5,0.012014,V5
6,0.010816,V6
7,0.021295,V7
8,0.011167,V8
9,0.026577,V9


In [ ]:
features.sort_values(by=['imp'], ascending=False).nlargest(8, 'imp')

,imp,variable
14,0.147299,V14
17,0.125091,V17
12,0.102777,V12
11,0.079358,V11
10,0.073989,V10
16,0.069491,V16
4,0.062904,V4
3,0.048322,V3


In [ ]:
data = data[(features.sort_values(by=['imp'], ascending=False).nlargest(8, 'imp')['variable']).append(pd.Series(['Class']), ignore_index=True)]

print(data.shape)

data.head(10)

(284807, 9)


,V14,V17,V12,V11,V10,V16,V4,V3,Class
0,-0.311169,0.207971,-0.617801,-0.551600,0.090794,-0.470401,1.378155,2.536347,0
1,-0.143772,-0.114805,1.065235,1.612727,-0.166974,0.463917,0.448154,0.166480,0
2,-0.165946,1.109969,0.066084,0.624501,0.207643,-2.890083,0.379780,1.773209,0
3,-0.287924,-0.684093,0.178228,-0.226487,-0.054952,-1.059647,-0.863291,1.792993,0
4,-1.119670,-0.237033,0.538196,-0.822843,0.753074,-0.451449,0.403034,1.548718,0
5,-0.137134,-0.058133,0.359894,1.341262,-0.371407,0.401726,-0.168252,1.141109,0
6,0.167372,0.002821,-0.153826,-1.416907,-0.099254,-0.443587,1.202613,0.045371,0
7,-1.323865,-1.222127,0.291474,-0.619468,1.249376,-0.076127,-0.492199,1.074380,0
8,0.074355,-0.499768,-0.110452,-0.705117,-0.410430,-0.210077,-0.271526,-0.113192,0
9,-0.443523,-0.540980,0.836390,1.017614,-0.366846,0.739453,-0.222187,1.044367,0


In [ ]:
data.drop(axis=0, index=list(data.index[: 7]), inplace=True)

data.shape

(284800, 9)

In [ ]:
x = np.array(data[data.columns[:-1]])
y = np.array(data['Class'])

#print(x, y)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=44)

In [ ]:
kfold = KFold(n_splits=10, random_state=44)
cv_result = cross_val_score(rfc, x, y, scoring='roc_auc', cv=kfold)
result_list = "%s: %f (%f)" % ("RandomForestClassifier: ", cv_result.mean(), cv_result.std())

print(result_list)

RandomForestClassifier: : 0.909969 (0.040462)


In [ ]:
lgbm = LGBMClassifier(n_estimators=10, learning_rate=.0001, class_weight='balanced', random_state=44)

In [ ]:
cv_lgbm = cross_val_score(lgbm, x, y, scoring='roc_auc', cv=kfold)

result_lgbm = "%s: %f (%f)" % ('lgbm', cv_lgbm.mean(), cv_lgbm.std())

print(result_lgbm)

lgbm: 0.925343 (0.028400)


In [ ]:
vote = VotingClassifier(estimators=[('Random Forest', rfc), ('LightGBM', lgbm)], voting='soft')

In [ ]:
roc_auc_score = cross_val_score(vote, x, y, cv=kfold, scoring='roc_auc')

print('roc_auc_score-- Mean %s STD %s' % (np.mean(roc_auc_score), np.std(roc_auc_score)))

roc_auc_score-- Mean 0.9369993135877452 STD 0.027906973493664715
